# Needed Libraries

In [1]:
import datetime as dt
import numpy as np
import matplotlib.pyplot as plt
import ta
import pandas as pd
import pandas_datareader.data as web
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

Using TensorFlow backend.


# Dataframe Creation

In [2]:
#vanguard total bond market etf
ticker = 'bnd'
end = dt.datetime.now()
start = end - dt.timedelta(days = 365*15)

def taStock(ticker,start,end):
    df = web.DataReader(ticker.upper(),'yahoo',start,end).reset_index()
    data = ta.add_all_ta_features(df, "Open", "High", "Low", "Close", "Volume", fillna=True)
    data['Daily Return'] = (data['Adj Close'].pct_change())*100
    data['target'] = int(False)
    data.loc[data['Daily Return'] > 0, 'target']=int(True)
    data = data.dropna() 
    return data

data = taStock(ticker,start,end)
data = data[['target','High','Low','Volume','volume_adi', 'volume_obv', 'volume_cmf', 
        'volume_fi', 'momentum_mfi','momentum_ao', 'momentum_kama', 'momentum_roc',
        'volume_em', 'volume_sma_em', 'volume_vpt', 'volume_nvi', 'volume_vwap',
        'volatility_atr', 'volatility_bbm', 'volatility_bbh', 'volatility_bbl',
        'volatility_bbw', 'volatility_bbp', 'volatility_bbhi',
        'volatility_bbli', 'volatility_kcc', 'volatility_kch', 'volatility_kcl',
        'volatility_kcw', 'volatility_kcp', 'volatility_kchi',
        'volatility_kcli', 'volatility_dcl', 'volatility_dch', 'trend_macd',
        'trend_macd_signal', 'trend_macd_diff', 'trend_sma_fast',
        'trend_sma_slow', 'trend_ema_fast', 'trend_ema_slow', 'trend_adx',
        'trend_adx_pos', 'trend_adx_neg', 'trend_vortex_ind_pos',
        'trend_vortex_ind_neg', 'trend_vortex_ind_diff', 'trend_trix',
        'trend_mass_index', 'trend_cci', 'trend_dpo', 'trend_kst',
        'trend_kst_sig', 'trend_kst_diff', 'trend_ichimoku_conv',
        'trend_ichimoku_base', 'trend_ichimoku_a', 'trend_ichimoku_b',
        'trend_visual_ichimoku_a', 'trend_visual_ichimoku_b', 'trend_aroon_up',
        'trend_aroon_down', 'trend_aroon_ind', 'trend_psar_up',
        'trend_psar_down', 'trend_psar_up_indicator',
        'trend_psar_down_indicator', 'momentum_rsi', 'momentum_tsi',
        'momentum_uo', 'momentum_stoch', 'momentum_stoch_signal', 'momentum_wr'
        ]]

# Splitting data between training and testing

In [3]:
train_size = int(0.8*len(data))
features = data.drop(columns = ['target'])
targets = data['target']
X_train,X_test = features.values[:train_size,:],features.values[train_size:,:]
y_train,y_test = targets.values[:train_size],targets.values[train_size:]

# Build ANN Model, Compile, and Training

In [4]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(1024,activation = tf.nn.leaky_relu),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(512,activation = tf.nn.leaky_relu),
    tf.keras.layers.Dropout(0.2),
    #sigmoid is the best to use in the output layer for a binary classification artificial neural network
    tf.keras.layers.Dense(1,activation='sigmoid')
])
#class weights => these are needed for a stock that historically is known for a linear upward trend
up = len(y_train[y_train>0])
down = len(y_train)-up
total = len(y_train)
weight_for_up = total / (2*up)
weight_for_down = total / (2*down)
class_weight = {0:weight_for_down, 1:weight_for_up}
print(class_weight)

# Compile with optimizer, loss fxn, and metric (accuracy is good for classification models)
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

#preprocessing 
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
# Fitting to the training set
model.fit(X_train_scaled,y_train,class_weight=class_weight,batch_size=32,epochs=10,validation_data=(X_test,y_test))

{0: 1.067945643485212, 1: 0.94018296973962}
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train on 2672 samples, validate on 669 samples
Epoch 1/10
2672/2672 [==============================] - 2s 711us/sample - loss: 0.5851 - accuracy: 0.7133 - val_loss: 1884807.6076 - val_accuracy: 0.5486
Epoch 2/10
2672/2672 [==============================] - 1s 327us/sample - loss: 0.4698 - accuracy: 0.7713 - val_loss: 4190897.7347 - val_accuracy: 0.5486
Epoch 3/10
2672/2672 [==============================] - 1s 295us/sample - loss: 0.4255 - accuracy: 0.8016 - val_loss: 4546899.1248 - val_accuracy: 0.5486
Epoch 4/10
2672/2672 [==============================] - 1s 343us/sample - loss: 0.3858 - accuracy: 0.8234 - val_loss: 3342858.9634 - val_accuracy: 0.4514
Epoch 5/10
2672/2672 [==============================] - 1s 319us/sample - loss: 0.3907 - accuracy: 0.8249 - val_loss: 4102384.7272 - val_accuracy: 0.5486
Epoch 6/10
2672/2672 [==============================] - 1s 333us/sample - loss: 0.3707 - 

In [5]:
X_test_scaled = scaler.transform(X_test)
model.evaluate(X_test_scaled,y_test)

669/669 [==============================] - 0s 341us/sample - loss: 1.0418 - accuracy: 0.8117


[1.0417886093949345, 0.8116592]